In [1]:
import boto3
import pickle as pkl
import pandas as pd
import csv
import time
bucket = 'youtube-overlaps'

In [2]:
def see_all_files_s3(bucket):
    s3 = boto3.resource('s3')
    s3_bucket = s3.Bucket(bucket)
    files = []
    for obj in s3_bucket.objects.all():
        files.append(obj.key)
    return files

def load_pkl_obj_s3(file_name, bucket):
    try:
        s3 = boto3.resource('s3')
        obj = pkl.loads(s3.Bucket(bucket).Object(file_name).get()['Body'].read())
    except:
        time.sleep(5)
        return load_pkl_obj_s3(file_name, bucket)
        
    return obj

In [8]:
commenter_files = see_all_files_s3(bucket)
channel_file_dict = {}
for file_name in commenter_files:
    prefix_len = 9
    suffix_len = 17
    channel_name = file_name[prefix_len:-suffix_len]
    channel_file_dict[channel_name] = file_name

len(channel_file_dict)

2441

In [9]:
master_overlap_dict = {}
for primary_channel, _file in channel_file_dict.items():
    master_overlap_dict.update(load_pkl_obj_s3(_file, bucket))

print(len(master_overlap_dict))

2363


In [10]:
channel_df = pd.read_csv('Top20kYoutubeChannels.csv')

In [11]:
def create_edges_csv(overlap_dict):
    with open('youtube_edges.csv', 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Source', 'Target', 'Weight'])
        for primary_channel, primary_overlap_dict in master_overlap_dict.items():
            for comparison_channel, shared_overlap in primary_overlap_dict.items():
                writer.writerow([primary_channel, comparison_channel, shared_overlap])

def create_labels_csv(overlap_dict, channel_df):
    with open('youtube_labels.csv', 'w', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Id', 'Label', 'Count'])
        for primary_channel in overlap_dict:
            display_name = channel_df[channel_df['username'] == primary_channel].iloc[0]['displayname']
            subscriber_count = channel_df[channel_df['username'] == primary_channel].iloc[0]['subscribers']
            writer.writerow([primary_channel, display_name, subscriber_count])

In [12]:
create_edges_csv(master_overlap_dict)
create_labels_csv(master_overlap_dict, channel_df)